In [5]:
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torchtext
import tqdm
import sys

C:\Users\Tyrone\anaconda3\envs\nlp\python.exe


In [6]:
train_iter = torchtext.datasets.IMDB(split='train')

In [8]:
tokenizor = torchtext.data.utils.get_tokenizer('basic_english')

In [9]:
tokens = []
for label, line in train_iter:
    tokens += tokenizor(line)

In [10]:
from collections import Counter, OrderedDict
# from torchtext.vocab import Vocab

counter = Counter(tokens) # unsorted dict
sorted_tuples = sorted(counter.items(), key=lambda x: x[1], reverse=True)
ordered_dict = OrderedDict(sorted_tuples)
Vocab = torchtext.vocab.vocab(ordered_dict, min_freq=10)

# adding <unk> token and default index
unk_token = '<unk>'
default_index = 0
if unk_token not in Vocab:
    Vocab.insert_token(unk_token, default_index)
Vocab.set_default_index(default_index)

# adding <pad> token
Vocab.insert_token('<pad>', 1)

In [23]:
from torch.utils.data import DataLoader, Dataset
from torch.nn.utils.rnn import pad_sequence
import random

text_transform = lambda x: [Vocab[token] for token in tokenizor(x)]
label_transform = lambda x: 1 if x == 'pos' else 0

def collate_batch(batch):
    label_list, text_list = [], []
    for label, text in batch:
        text_list.append(torch.tensor(text_transform(text)))
        label_list.append(label_transform(label))
    
    return torch.tensor(label_list), pad_sequence(text_list, padding_value=Vocab['<pad>'])

train_iter, test_iter = torchtext.datasets.IMDB(split=('train', 'test'))

class dataset_from_iter(Dataset):
    def __init__(self, train_iter):
        self.labels, self.text = [], []
        for label, line in train_iter:
            self.labels.append(label)
            self.text.append(line)
    
    def __getitem__(self, index):
        return self.labels[index], self.text[index]
    
    def __len__(self):
        return len(self.labels)

train_dataset = dataset_from_iter(train_iter)

In [47]:
batch_size = 64

n_train = len(train_dataset)
split = n_train // 3
indices = list(range(n_train))
random.shuffle(indices)
train_sampler = torch.utils.data.sampler.SubsetRandomSampler(indices[split:])
valid_sampler = torch.utils.data.sampler.SubsetRandomSampler(indices[:split])

train_dataloader = DataLoader(train_dataset, batch_size=batch_size, sampler=train_sampler, collate_fn=collate_batch)
valid_dataloader = DataLoader(train_dataset, batch_size=batch_size, sampler=valid_sampler, collate_fn=collate_batch)
test_dataloader = DataLoader(test_iter, batch_size=batch_size, collate_fn=collate_batch)

In [43]:
class LSTM(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, n_layers, bidirectional, dropout_rate, pad_index):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=pad_index)
        self.rnn = nn.LSTM(embedding_dim, hidden_dim, n_layers, bidirectional=bidirectional, dropout=dropout_rate)
        self.fc = nn.Linear(hidden_dim*2 if bidirectional else hidden_dim, output_dim)
        self.dropout = nn.Dropout(dropout_rate)
    
    def forward(self, text):
        embedded = self.dropout(self.embedding(text))
        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, lengths=[embedded.size(1) for i in range(embedded.size(0))])

        _, (hidden, _) = self.rnn(packed_embedded)

        if self.rnn.bidirectional:
            hidden = self.dropout(torch.cat([hidden[-1], hidden[-2]], dim=-1))
            # [batch_size, hidden_dim * 2]
        else:
            hidden = self.dropout(hidden[-1])
            # [batch_size, hidden_dim]
        
        predictions = self.fc(hidden)

        return predictions


In [44]:
vocab_size = len(Vocab)
embedding_dim = 300
hidden_dim = 300
output_dim = 1
n_layers = 2
bidirectional = True
dropout_rate = 0.5

model = LSTM(vocab_size, embedding_dim, hidden_dim, output_dim, n_layers, bidirectional, dropout_rate, pad_index=Vocab.get_stoi()['<pad>'])

In [46]:
# TODO
# add pretrained model

lr = 5e-4
optimizer = torch.optim.Adam(model.parameters(), lr)
criterion = nn.CrossEntropyLoss()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = model.to(device)
criterion = criterion.to(device)

In [52]:
def binary_accuracy(preds, y):
    rounded_preds = torch.round(preds)
    correct = (rounded_preds == y).float()
    acc = correct.sum() / len(correct)

    return acc

In [54]:
def train(data_loader, model, criterion, optimizer, device):
    model.train()
    epoch_losses = []
    epoch_accs = []

    for labels, text in tqdm.tqdm(data_loader, desc='training...', file=sys.stdout):
        labels = labels.to(device)
        text = text.to(device)

        predictions = model(text)
        loss = criterion(predictions, labels)
        acc = binary_accuracy(predictions, labels)

        epoch_losses.append(loss.item)
        epoch_accs.append(acc.item)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    return epoch_losses, epoch_accs

In [55]:
def evaluate(data_loader, model, criterion, device):
    model.eval()
    losses = []
    accs = []

    with torch.no_grad():
        for labels, text in tqdm.tqdm(data_loader, desc='evaluating...', file=sys.stdout):
            labels = labels.to(device)
            text = text.to(device)

            predictions = model(text)
            loss = criterion(predictions, labels)
            acc = binary_accuracy(predictions, labels)

            losses.append(loss)
            accs.append(acc)
        
        return losses, accs

In [ ]:
n_epoch = 10
best_valid_loss = float('inf')
train_losses = []
train_accs = []
valid_losses = []
valid_accs = []

for epoch in range(n_epoch):
    train_loss, train_acc = train(train_dataloader, model, criterion, optimizer, device)
    valid_loss, valid_acc = evaluate(valid_dataloader, model, criterion, device)

    train_losses.extend(train_loss)
    train_accs.extend(train_acc)
    valid_losses.extend(valid_loss)
    valid_accs.extend(valid_acc)

    train_loss_mean = np.mean(train_loss)
    train_acc_mean = np.mean(train_acc)
    valid_loss_mean = np.mean(valid_loss)
    valid_acc_mean = np.mean(valid_acc)

    if valid_loss_mean < best_valid_loss:
        best_valid_loss = valid_loss_mean
        torch.save(model.state_dict(), 'best_model')
    
    print(f'epoch: {epoch+1:02}')
    print(f'train loss: {train_loss_mean:.3f}, train acc: {train_acc_mean:.3f}')
    print(f'valid loss: {train_loss_mean:.3f}, valid acc: {train_acc_mean:.3f}')